In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from typing import Any, Dict, List, Optional, Tuple

from dotenv import load_dotenv
load_dotenv()
from setup_path import add_src_to_path
add_src_to_path()

from awear_neuroscience.data_extraction.constants import (
    WAVEFORM_KEY, SAMPLING_RATE, FIELD_KEYS
)

from awear_neuroscience.data_extraction.firestore_loader import  process_eeg_records
from awear_neuroscience.pipeline.preprocess import process_long_df, extract_features_from_long_df, process_features
from awear_neuroscience.statistical_analysis.statistical_tests import compare_session_types
from awear_neuroscience.data_extraction.firestore_loader import get_selreport_data

In [ ]:
import pandas as pd

features_dfs = pd.read_csv("data/focus_sessions_labels_eeg_features.csv")
features_dfs.head()

In [ ]:

features_dfs['stress_index'] = (features_dfs['beta_db_fil']+features_dfs['gamma_db_fil']) / (features_dfs['theta_db_fil']+features_dfs['alpha_db_fil'])
features_dfs['stress_index_2']= (features_dfs['beta2_db_fil']+features_dfs['gamma2_db_fil']) / (features_dfs['theta_db_fil']+features_dfs['alpha_db_fil'])
features_dfs['beta_over_theta_gamma']= (features_dfs['beta_db_fil']) / (features_dfs['theta_db_fil']+features_dfs['gamma_db_fil'])
epsilon = 1e-6

features_dfs['stress_index_log'] = np.log((features_dfs['beta_db_fil']+features_dfs['gamma_db_fil']) / (features_dfs['theta_db_fil']+features_dfs['alpha_db_fil']+ epsilon))
features_dfs['stress_index_2_log']= np.log((features_dfs['beta2_db_fil']+features_dfs['gamma2_db_fil']) / (features_dfs['theta_db_fil']+features_dfs['alpha_db_fil']+ epsilon))
features_dfs['beta_over_theta_gamma_log']= np.log((features_dfs['beta_db_fil']) / (features_dfs['theta_db_fil']+features_dfs['gamma_db_fil']+ epsilon))



In [ ]:
FEATURES = ['delta_db_fil',
       'theta_db_fil', 'alpha_db_fil', 'beta_db_fil', 'gamma_db_fil',
       'alpha1_db_fil', 'alpha2_db_fil', 'beta1_db_fil', 'beta2_db_fil',
       'beta3_db_fil', 'gamma1_db_fil', 'gamma2_db_fil',
       'theta_beta_ratio_fil_db', 'theta_alpha_ratio_fil_db',
       'beta_alpha_ratio_fil_db', 'engagement_index_fil_db',
       'focus_index_fil_db', 'stress_index', 'stress_index_2', 'beta_over_theta_gamma', 'stress_index_log', 'stress_index_2_log', 'beta_over_theta_gamma_log']

Step 2: Test for Normality

- If **p > 0.05** for both Calm and Stressed: assume normality
- Otherwise: use non-parametric test

In [ ]:
from scipy.stats import shapiro
p = shapiro(calm_values).pvalue


### Step 3: Hypothesis Testing

### If normality confirmed (both groups):

- Use **one-way ANOVA (F-test)**:

    

### If normality NOT confirmed or unequal variances:

- Use **Mann-Whitney U test**:


**Use p_value < 0.01** as significance threshold. If p_value < 0.01 the null hypothesis can be rejected

In [ ]:
from scipy.stats import f_oneway, mannwhitneyu
f_stat, p_value = f_oneway(calm_values, stressed_values)
stat, p_value = mannwhitneyu(calm_values, stressed_values)

### Step 4: Effect Size (Cohen’s d)

**Cohen’s d** is a standardized measure of **effect size**. It tells you **how big the difference is between two groups**, in units of standard deviation.


Effect Size	Cohen’s d
Small	0.2
Medium	0.5
Large	0.8+

In [ ]:
def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(((nx - 1)*np.std(x, ddof=1)**2 + (ny - 1)*np.std(y, ddof=1)**2) / (nx + ny - 2))
    return (np.mean(x) - np.mean(y)) / pooled_std


Step 5: Bootstrap 95% Confidence Interval for Effect Size

In [ ]:
def bootstrap_cohens_d(x, y, n_iter=10000):
    d_values = []
    for _ in range(n_iter):
        x_sample = np.random.choice(x, size=len(x), replace=True)
        y_sample = np.random.choice(y, size=len(y), replace=True)
        d_values.append(cohens_d(x_sample, y_sample))
    ci_lower = np.percentile(d_values, 2.5)
    ci_upper = np.percentile(d_values, 97.5)
    return np.mean(d_values), (ci_lower, ci_upper)


Interpretation Summary

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import f_oneway, mannwhitneyu

def ecdf_with_ci(data, alpha=0.05):
    n = len(data)
    epsilon = np.sqrt(np.log(2 / alpha) / (2 * n))
    x = np.sort(data)
    y = np.arange(1, n + 1) / n
    lower = np.maximum(y - epsilon, 0)
    upper = np.minimum(y + epsilon, 1)
    return x, y, lower, upper

def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(((nx - 1)*np.std(x, ddof=1)**2 + (ny - 1)*np.std(y, ddof=1)**2) / (nx + ny - 2))
    return (np.mean(x) - np.mean(y)) / pooled_std

def plot_ecdf_with_ci(calm_vals, stressed_vals, feature_name, email, stats, save_path=None, show=True):
    x_calm, y_calm, lower_calm, upper_calm = ecdf_with_ci(calm_vals)
    x_stressed, y_stressed, lower_stressed, upper_stressed = ecdf_with_ci(stressed_vals)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=x_calm, y=y_calm, name='Calm', line=dict(color='blue')))
    fig.add_trace(go.Scatter(
        x=np.concatenate([x_calm, x_calm[::-1]]),
        y=np.concatenate([upper_calm, lower_calm[::-1]]),
        fill='toself', fillcolor='rgba(0, 0, 255, 0.1)',
        line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", name='Calm CI'
    ))
    fig.add_trace(go.Scatter(x=x_stressed, y=y_stressed, name='Stressed', line=dict(color='orange')))
    fig.add_trace(go.Scatter(
        x=np.concatenate([x_stressed, x_stressed[::-1]]),
        y=np.concatenate([upper_stressed, lower_stressed[::-1]]),
        fill='toself', fillcolor='rgba(255, 165, 0, 0.1)',
        line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", name='Stressed CI'
    ))

    annotation_text = (f"Cohen's d = {stats['cohens_d']:.3f}<br>"
                       f"ANOVA p = {stats['anova_p']:.3g}<br>"
                       f"MW U p = {stats['mw_p']:.3g}")
    fig.add_annotation(
        text=annotation_text,
        xref="paper", yref="paper",
        x=0.01, y=0.99, showarrow=False,
        align="left", font=dict(size=12)
    )

    fig.update_layout(
        title=f"ECDF with CI: {feature_name} for {email}",
        xaxis_title=feature_name,
        yaxis_title="ECDF",
        template="simple_white"
    )

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        fig.write_image(save_path, format='png', scale=2)

    if show:
        fig.show()


# === Main loop ===
emails = df['email'].unique()
results = []

for email in emails:
    calm_df = df[(df['focus_type'] == 'calm') & (df['email'] == email)]
    stressed_df = df[(df['focus_type'] == 'stressed') & (df['email'] == email)]
    for feature in FEATURES:
        calm_vals = calm_df[feature].dropna().values
        stressed_vals = stressed_df[feature].dropna().values

        if len(calm_vals) < 4 or len(stressed_vals) < 4:
            continue

        # Compute stats
        try:
            anova_p = f_oneway(calm_vals, stressed_vals).pvalue
        except:
            anova_p = np.nan
        try:
            mw_p = mannwhitneyu(calm_vals, stressed_vals, alternative='two-sided').pvalue
        except:
            mw_p = np.nan
        try:
            d = cohens_d(calm_vals, stressed_vals)
        except:
            d = np.nan

        stats = {'email': email, 'feature': feature, 'anova_p': anova_p, 'mw_p': mw_p, 'cohens_d': d}
        results.append(stats)

        # Plot with stats in legend
        plot_ecdf_with_ci(calm_vals, stressed_vals, feature_name=feature, email=email, stats=stats, save_path='plots/', show=False)

# === Convert to DataFrame summary ===
summary_df = pd.DataFrame(results)
summary_df = summary_df[['email', 'feature', 'anova_p', 'mw_p', 'cohens_d']]


In [ ]:
def label_effect_size(d):
    if pd.isna(d):
        return 'NA'
    elif abs(d) >= 0.8:
        return 'Large'
    elif abs(d) >= 0.5:
        return 'Medium'
    elif abs(d) >= 0.2:
        return 'Small'
    else:
        return 'Negligible'

summary_df['effect_size_label'] = summary_df['cohens_d'].apply(label_effect_size)


In [ ]:
summary_df.to_csv("calm_vs_stressed_features_with_effect_size.csv", index=False)